In [1]:
import keras
import os
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# Load the model

In [2]:
from keras.models import load_model

model = load_model('cat_vs_dog.h5')

# Dataset

In [3]:
dataset_path = "./dataset"
training_set = dataset_path + "/train"
validation_set =dataset_path + "/val"

In [4]:
print("Number of cat images in the training set:", len(os.listdir(training_set + "/cat")))

Number of cat images in the training set: 10000


In [5]:
print("Number of dog images in the training set:", len(os.listdir(training_set + "/dog")))

Number of dog images in the training set: 10000


In [6]:
print("Number of cat images in the validation set:", len(os.listdir(validation_set + "/cat")))

Number of cat images in the validation set: 2500


In [7]:
print("Number of dog images in the validation set:", len(os.listdir(validation_set + "/dog")))

Number of dog images in the validation set: 2500


# Model architecture

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (5, 5), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 128)       147584    
__________

# Cost function, optimizer, ...

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-3),
              metrics=['acc'])

# Data generator

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# validation data should not be augmented!

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        training_set,
        target_size=(150, 150),
        batch_size=64,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_set,
        target_size=(150, 150),
        batch_size=64,
        class_mode='binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


# Training

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=312,
      epochs=20,
      validation_data=validation_generator,
      validation_steps=78)

# Plot accuracy and loss

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

# Save the model

In [ ]:
model.save('cat_vs_dog.h5')

# Train more with lower learning rate

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=312,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=78)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training Accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()

plt.show()

# Save the model

In [ ]:
model.save('cat_vs_dog.h5')

# Evaluate the model

In [10]:
test_loss, test_acc = model.evaluate_generator(validation_generator, steps=20)
print('Accuracy on Validation Set:', test_acc)

Accuracy on Validation Set: 0.915625
